#### Load the dataset.

In [1]:

import time
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample

df = pd.read_csv("../dataset/original-noneeg-dataset.csv", sep="|", dtype = {"hr": "float64", "label": "int8"})


#### Display the dataset size.

In [2]:
df.shape

(41992, 2)

#### Define a helper function to convert the set data to the required format to perform the undersampling.

In [3]:

number_of_steps = 30

def build_time_window_structure(df):
    # Splits the dataset into "time windows" to be used as a time series list.
    # The function groups each 30 dataset records (CSV lines) into one record.
    # Each record contains 30 steps and each step contains 1 feature value.
    # Parameters:
    #    df: Dataframe to be splitted.
    # Return:
    #    First list contains all time windows.
    #    Second list contains all target values.
    print("\nStarting build_time_window_structure function.")
    initial_line_number = 0
    first_feat_index = 0
    last_feat_index = 1
    X_array = []
    y_array = []
    while initial_line_number < len(df["label"]):
        target_value = df["label"][initial_line_number]
        sub_matrix = df.iloc[initial_line_number : (initial_line_number + number_of_steps), first_feat_index : last_feat_index]
        sub_matrix_values = sub_matrix.values
        new_line = sub_matrix_values.flatten()
        size_diff = number_of_steps - len(new_line)
        if size_diff > 0:
            last_value = new_line[len(new_line) - 1]
            new_line = np.append(new_line, [last_value] * size_diff)
        X_array.append(new_line)
        y_array.append(target_value)
        initial_line_number += number_of_steps
    print("Quantity of samples (features) => ", len(X_array))
    print("Quantity os samples (labels) => ", len(y_array))
    print("Finishing build_time_window_structure function.")
    return X_array, y_array


#### Perform undersampling for balancing the dataset.

In [4]:

# Correct dataset imbalance through undersampling.
print("\nStarting undersampling process.")
X_list, y_list = build_time_window_structure(df)
rus = RandomUnderSampler(random_state = 42)
X_arr = np.array(X_list)
y_arr = np.array(y_list)
X_resampled, y_resampled = rus.fit_resample(X_arr, y_arr)
print("\nQuantity of resampled samples => ", len(y_resampled))



Starting undersampling process.

Starting build_time_window_structure function.
Quantity of samples (features) =>  1400
Quantity os samples (labels) =>  1400
Finishing build_time_window_structure function.

Quantity of resampled samples =>  800


#### GridSearch to evaluate Random Forest hyperparameters.

In [6]:

model = RandomForestClassifier(random_state = 42, n_jobs = 4)
params = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, 10, 15, 21],
    'criterion': ['gini', 'entropy']
}
strat_k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
grid_search_cv = GridSearchCV(model, cv = strat_k_fold, param_grid = params, n_jobs = 4, verbose = 5)

# Train the Random Forest model and show the best hyperparameter values.
start_time = time.time()
print("\nStarting training...")
grid_search_cv.fit(X_resampled, y_resampled)
elapsed_seconds = time.time() - start_time
print("\nTime taken for training: ", time.strftime("%H:%M:%S", time.gmtime(elapsed_seconds)))
print("\nBest values:")
print(grid_search_cv.best_params_)



Starting training...
Fitting 10 folds for each of 36 candidates, totalling 360 fits

Time taken for training:  00:04:18

Best values:
{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 300}


#### Train the model using cross-validation (10 fold) and display metrics.

In [7]:

# Normalize data using PowerTransformer method.
pt = PowerTransformer()

strat_k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

# Train the KNN model and show the best hyperparameter values.
start_time = time.time()
print("\nStarting training...\n")
train_accuracy_by_fold = []
test_accuracy_by_fold = []
y_predclass_for_report = []
y_testclass_for_report = []
fold_number = 1
for train_index, test_index in strat_k_fold.split(X_resampled, y_resampled):
    print("Training fold {}".format(fold_number))
    model = RandomForestClassifier(random_state = 42, n_jobs = 4, n_estimators = 300, max_depth = 5, criterion = "gini")
    X_train_scaled = pt.fit_transform(X_resampled[train_index])
    X_test_scaled = pt.transform(X_resampled[test_index])
    model.fit(X_train_scaled, y_resampled[train_index])
    train_pred_classes = model.predict(X_train_scaled)
    test_pred_classes = model.predict(X_test_scaled)
    train_accuracy = accuracy_score(y_resampled[train_index], train_pred_classes)
    test_accuracy = accuracy_score(y_resampled[test_index], test_pred_classes)
    train_accuracy_by_fold.append(train_accuracy)
    test_accuracy_by_fold.append(test_accuracy)
    y_predclass_for_report.extend(test_pred_classes)
    y_testclass_for_report.extend(y_resampled[test_index])
    fold_number += 1

elapsed_seconds = time.time() - start_time
print("\nTime taken for training: ", time.strftime("%H:%M:%S", time.gmtime(elapsed_seconds)))
print("\n")

# Show metrics.
for i in range(len(train_accuracy_by_fold)):
    print("Fold {} - Train Accuracy {:.4f} - Test Accuracy {:.4f}".format((i + 1),
                            train_accuracy_by_fold[i], test_accuracy_by_fold[i]))
print("\nMean Train Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(train_accuracy_by_fold), np.std(train_accuracy_by_fold)))
print("Mean Test Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(test_accuracy_by_fold), np.std(test_accuracy_by_fold)))

print("\nEvaluate other metrics:")
print(classification_report(y_testclass_for_report, y_predclass_for_report))



Starting training...

Training fold 1
Training fold 2
Training fold 3
Training fold 4
Training fold 5
Training fold 6
Training fold 7
Training fold 8
Training fold 9
Training fold 10

Time taken for training:  00:00:18


Fold 1 - Train Accuracy 0.6486 - Test Accuracy 0.4625
Fold 2 - Train Accuracy 0.6306 - Test Accuracy 0.4625
Fold 3 - Train Accuracy 0.6264 - Test Accuracy 0.4750
Fold 4 - Train Accuracy 0.6292 - Test Accuracy 0.4875
Fold 5 - Train Accuracy 0.6472 - Test Accuracy 0.4375
Fold 6 - Train Accuracy 0.6389 - Test Accuracy 0.4500
Fold 7 - Train Accuracy 0.6319 - Test Accuracy 0.5125
Fold 8 - Train Accuracy 0.6375 - Test Accuracy 0.4875
Fold 9 - Train Accuracy 0.6306 - Test Accuracy 0.3750
Fold 10 - Train Accuracy 0.6347 - Test Accuracy 0.4125

Mean Train Accuracy: 0.6356 - Std: 0.0071 
Mean Test Accuracy: 0.4562 - Std: 0.0380 

Evaluate other metrics:
              precision    recall  f1-score   support

           0       0.44      0.27      0.33       200
           1     

#### Train the model using cross-validation (10 fold) and display metrics.
- Data augmentation (5x).

In [8]:

# Normalize data using PowerTransformer method.
pt = PowerTransformer()

strat_k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

# Train the KNN model and show the best hyperparameter values.
start_time = time.time()
print("\nStarting training...\n")
train_accuracy_by_fold = []
test_accuracy_by_fold = []
y_predclass_for_report = []
y_testclass_for_report = []
fold_number = 1
for train_index, test_index in strat_k_fold.split(X_resampled, y_resampled):
    print("Training fold {}".format(fold_number))
    model = RandomForestClassifier(random_state = 42, n_jobs = 4, n_estimators = 300, max_depth = 5, criterion = "gini") 
    # Data augmentation (5x).
    X_oversampled, y_oversampled = resample(X_resampled[train_index],
                                            y_resampled[train_index],
                                            replace = True,
                                            n_samples = 4000,
                                            stratify = y_resampled[train_index],
                                            random_state = 42)
    X_train_scaled = pt.fit_transform(X_oversampled)
    X_test_scaled = pt.transform(X_resampled[test_index])
    model.fit(X_train_scaled, y_oversampled)
    train_pred_classes = model.predict(X_train_scaled)
    test_pred_classes = model.predict(X_test_scaled)
    train_accuracy = accuracy_score(y_oversampled, train_pred_classes)
    test_accuracy = accuracy_score(y_resampled[test_index], test_pred_classes)
    train_accuracy_by_fold.append(train_accuracy)
    test_accuracy_by_fold.append(test_accuracy)
    y_predclass_for_report.extend(test_pred_classes)
    y_testclass_for_report.extend(y_resampled[test_index])
    fold_number += 1

elapsed_seconds = time.time() - start_time
print("\nTime taken for training: ", time.strftime("%H:%M:%S", time.gmtime(elapsed_seconds)))
print("\n")

# Show metrics.
for i in range(len(train_accuracy_by_fold)):
    print("Fold {} - Train Accuracy {:.4f} - Test Accuracy {:.4f}".format((i + 1),
                            train_accuracy_by_fold[i], test_accuracy_by_fold[i]))
print("\nMean Train Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(train_accuracy_by_fold), np.std(train_accuracy_by_fold)))
print("Mean Test Accuracy: {:.4f} - Std: {:.4f} ".format(np.mean(test_accuracy_by_fold), np.std(test_accuracy_by_fold)))

print("\nEvaluate other metrics:")
print(classification_report(y_testclass_for_report, y_predclass_for_report))



Starting training...

Training fold 1
Training fold 2
Training fold 3
Training fold 4
Training fold 5
Training fold 6
Training fold 7
Training fold 8
Training fold 9
Training fold 10

Time taken for training:  00:00:25


Fold 1 - Train Accuracy 0.6743 - Test Accuracy 0.4250
Fold 2 - Train Accuracy 0.6715 - Test Accuracy 0.4375
Fold 3 - Train Accuracy 0.6617 - Test Accuracy 0.4625
Fold 4 - Train Accuracy 0.6810 - Test Accuracy 0.4875
Fold 5 - Train Accuracy 0.6973 - Test Accuracy 0.4375
Fold 6 - Train Accuracy 0.6705 - Test Accuracy 0.4500
Fold 7 - Train Accuracy 0.6687 - Test Accuracy 0.5125
Fold 8 - Train Accuracy 0.6505 - Test Accuracy 0.4750
Fold 9 - Train Accuracy 0.6793 - Test Accuracy 0.3500
Fold 10 - Train Accuracy 0.6655 - Test Accuracy 0.4250

Mean Train Accuracy: 0.6720 - Std: 0.0118 
Mean Test Accuracy: 0.4462 - Std: 0.0419 

Evaluate other metrics:
              precision    recall  f1-score   support

           0       0.39      0.26      0.31       200
           1     